## Test for KL Leibler Divergence

In [106]:
# %env

In [107]:
%pwd

'/Users/liamroy/Documents/Studies/Monash_31194990/PHD/Studies/Study_03/LLM_motion/llm_audio_testcase/stats'

In [108]:
import os

import pandas as pd
pd.set_option('future.no_silent_downcasting', True)

import numpy as np

from openpyxl import Workbook
from openpyxl import load_workbook

In [109]:
conditions = ["llm", "human"]
states = ["stuck", "accomplished", "progressing"]

for state in states:
    print(f"\n\n@@@@@@@@@@@@@@@ Kullback–Leibler Divergence For State: {state} @@@@@@@@@@@@@@@\n")

    # FIRST DO LLM COND:
    # Load the data from the Excel file, specifying the sheet name
    llm_file_path = './../llm_audio_rawdata.xlsx'  # Replace with your file path
    llm_data = pd.read_excel(llm_file_path,sheet_name='llm_' + state + '_00', usecols="D:F", nrows=80)

    mapping = {'A': 0, 'B': 1, 'C': 2}
    llm_data = llm_data.replace(mapping).infer_objects()

    # Convert data to numeric values, coercing errors (e.g., empty cells or non-numeric text will become NaN)
    llm_data = llm_data.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

    # Ensure the data values are between 0 and 2 as required
    assert llm_data.values.max() <= 2 and llm_data.values.min() >= 0, "Values out of range (0, 1, 2)"

    # Initialize a 3x3x3 numpy array to hold the histogram counts
    llm_data_histogram = np.zeros((3, 3, 3), dtype=int)

    # Iterate through each row in the DataFrame and bin the data
    for row in llm_data.itertuples(index=False):
        x, y, z = row  # Unpack the three-vector (each value will be 0, 1, or 2)
        llm_data_histogram[x, y, z] += 1

    print(f"LLM DATA |3x3x3 Histogram Matrix for {state}:")
    print(llm_data_histogram)


    # THEN DO HUMAN COND:
    # Load the data from the Excel file, specifying the sheet name
    human_file_path = './../llm_audio_rawdata.xlsx'  # Replace with your file path
    human_data = pd.read_excel(human_file_path,sheet_name='human_' + state + '_00', usecols="D:F", nrows=24)

    mapping = {'A': 0, 'B': 1, 'C': 2}
    human_data = human_data.replace(mapping).infer_objects()

    # Convert data to numeric values, coercing errors (e.g., empty cells or non-numeric text will become NaN)
    human_data = human_data.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

    # Ensure the data values are between 0 and 2 as required
    assert human_data.values.max() <= 2 and human_data.values.min() >= 0, "Values out of range (0, 1, 2)"

    # Initialize a 3x3x3 numpy array to hold the histogram counts
    human_data_histogram = np.zeros((3, 3, 3), dtype=int)

    # Iterate through each row in the DataFrame and bin the data
    for row in human_data.itertuples(index=False):
        x, y, z = row  # Unpack the three-vector (each value will be 0, 1, or 2)
        human_data_histogram[x, y, z] += 1

    print(f"HUMAN DATA | 3x3x3 Histogram Matrix for {state}:")
    print(human_data_histogram)


    # Then, flatten each 3x3x3 matrix into a 1D vector of length 27
    llm_vector = llm_data_histogram.flatten()
    human_vector = human_data_histogram.flatten()


    # Then, convert to probability distributions. Since KL Divergence operates on 
    # probability distributions, normalize each vector so that the sum of its elements is 1
    llm_prob = llm_vector / llm_vector.sum()
    human_prob = human_vector / human_vector.sum()

    # Next, add smoothing. Since some bins in B_prob are zero, add a small 
    # smoothing value (1e−9) to avoid division by zero in the KL Divergence calculation
    llm_prob = np.where(llm_prob == 0, 1e-9, llm_prob)
    human_prob = np.where(human_prob == 0, 1e-9, human_prob)  

    # Finally, compute KL Divergence using the formula:KL_divergence = np.sum(seriesA * np.log(seriesA / seriesB))
    # This quantifies how how much Series B diverges from Series A, capturing the "distance" between the two distributions. 
    # Lower values indicate greater similarity, meaning Series A is more contained within Series B.

    KL_divergence = np.sum(llm_prob * np.log(llm_prob / human_prob))
 
    print(f"\n >>> KL Divergence for the state {state} is: {KL_divergence:.3f}\n")





@@@@@@@@@@@@@@@ Kullback–Leibler Divergence For State: stuck @@@@@@@@@@@@@@@

LLM DATA |3x3x3 Histogram Matrix for stuck:
[[[69  0  0]
  [ 2  0  0]
  [ 9  0  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]]
HUMAN DATA | 3x3x3 Histogram Matrix for stuck:
[[[9 1 0]
  [1 0 0]
  [4 0 0]]

 [[2 0 0]
  [0 0 0]
  [1 0 0]]

 [[1 0 0]
  [0 0 0]
  [5 0 0]]]

 >>> KL Divergence for the state stuck is: 0.661



@@@@@@@@@@@@@@@ Kullback–Leibler Divergence For State: accomplished @@@@@@@@@@@@@@@

LLM DATA |3x3x3 Histogram Matrix for accomplished:
[[[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  2]
  [ 0  0 24]
  [ 0  0 33]]

 [[ 0  0  0]
  [ 0  0 12]
  [ 0  0  9]]]
HUMAN DATA | 3x3x3 Histogram Matrix for accomplished:
[[[0 0 3]
  [0 1 2]
  [0 1 1]]

 [[0 0 2]
  [0 0 2]
  [0 0 1]]

 [[0 0 2]
  [0 0 5]
  [1 1 2]]]

 >>> KL Divergence for the state accomplished is: 1.284



@@@@@@@@@@@@@@@ Kullback–Leibler Divergence For State: progressing @@@@@@@@@@@@@@@

LLM 